# **Launch Sites Locations Analysis with Folium**


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors that we could by analyzing the existing launch site locations.


## Objectives


This notebook contains the following:

*   Mark all launch sites on a map
*   Mark the success/failed launches for each site on the map
*   Calculate the distances between a launch site to its proximities


In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

## Mark all launch sites on a map


Add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [5]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


Create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [7]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [10]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='blue', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:'#d35400';"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

Add a circle for each launch site in data frame `launch_sites`


In [19]:
# Initialize the map
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
# For each launch site, add a Circle object based on its coordinate (Lat, Long) values. In addition, add Launch site name as a popup label
for index, site in launch_sites_df.iterrows():
    site_name = site['Launch Site']
    site_coordinate = [site['Lat'], site['Long']]

    # Create a yellow circle with a popup label
    circle = folium.Circle(site_coordinate, radius=1000, color='yellow', fill=True).add_child(folium.Popup(site_name))
    site_map.add_child(circle)

    # Create a marker with an icon showing the site name
    marker = folium.Marker(
        site_coordinate,
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % site_name,
        )
    )
    site_map.add_child(circle)
    site_map.add_child(marker)

site_map

# Mark the success/failed launches for each site on the map


Add the launch outcomes for each site, and see which sites have high success rates based upon the 'class' feature of the dataframe.


In [20]:
spacex_df.tail(10)

,Launch Site,Lat,Long,class
46,KSC LC-39A,28.573255,-80.646895,1
47,KSC LC-39A,28.573255,-80.646895,1
48,KSC LC-39A,28.573255,-80.646895,1
49,CCAFS SLC-40,28.563197,-80.576820,1
50,CCAFS SLC-40,28.563197,-80.576820,1
51,CCAFS SLC-40,28.563197,-80.576820,0
52,CCAFS SLC-40,28.563197,-80.576820,0
53,CCAFS SLC-40,28.563197,-80.576820,0
54,CCAFS SLC-40,28.563197,-80.576820,1
55,CCAFS SLC-40,28.563197,-80.576820,0


Create markers for all launch records.
If a launch was successful `(class=1)`, use a green marker and if a launch was failed, use a red marker `(class=0)`


Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


In [21]:
marker_cluster = MarkerCluster()


Create a new column in `launch_sites` dataframe called `marker_color` to store the marker colors based on the `class` value


In [25]:
# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')

spacex_df

,Launch Site,Lat,Long,class,marker_color
0,CCAFS LC-40,28.562302,-80.577356,0,red
1,CCAFS LC-40,28.562302,-80.577356,0,red
2,CCAFS LC-40,28.562302,-80.577356,0,red
3,CCAFS LC-40,28.562302,-80.577356,0,red
4,CCAFS LC-40,28.562302,-80.577356,0,red
5,CCAFS LC-40,28.562302,-80.577356,0,red
6,CCAFS LC-40,28.562302,-80.577356,0,red
7,CCAFS LC-40,28.562302,-80.577356,0,red
8,CCAFS LC-40,28.562302,-80.577356,0,red
9,CCAFS LC-40,28.562302,-80.577356,0,red


For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [27]:
# Add marker_cluster to current site_map
site_map.add_child(marker_cluster)

# for each row in spacex_df data frame
# create a Marker object with its coordinate
# and customize the Marker's icon property to indicate if this launch was successed or failed, 
for index, record in spacex_df.iterrows():
    # Create and add a Marker cluster to the site map
    marker = folium.Marker(
        location=[record['Lat'], record['Long']],
        icon=folium.Icon(color='white', icon_color=record['marker_color']),
    )
    marker_cluster.add_child(marker)

site_map

From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


# TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [28]:
# Add Mouse Position to get the coordinate (Lat, Long) for a mouse over on the map
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

Calculate the distance between the launch sites and various landmarks (railways, highways, coastlines, etc.)

In [29]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Find the coordinates of the closest coastline

In [38]:
# find coordinate of the closet coastline
coastline_lat = 28.56318
coastline_long = -80.56815
launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].values[0]
launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].values[0]
distance_coastline = calculate_distance(launch_site_lat, launch_site_long, coastline_lat, coastline_long)

distance_coastline

0.8469954937192601

In [44]:
# Create and add a folium.Marker on your selected closest coastline point on the map
# Display the distance between coastline point and launch site using the icon property 
coast_coordinate = [coastline_lat, coastline_long]
distance_marker = folium.Marker(
  coast_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_coastline),
      )
  )

site_map.add_child(distance_marker)
site_map

Draw a `PolyLine` between a launch site to the selected coastline point


In [49]:
# Create a `folium.PolyLine` object using the coastline coordinates and launch site coordinate
coordinates = [[coastline_lat, coastline_long], [launch_site_lat, launch_site_long]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Draw a line between a launch site to its closest city, railway, highway


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [53]:
# Create a marker with distance to a closest city, railway, highway, etc.
# Draw a line between the marker to the launch site
Kennedy_parkway_lat = 28.57336
Kennedy_parkway_long = -80.65553
KSC_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'KSC LC-39A', 'Lat'].values[0]
KSC_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'KSC LC-39A', 'Long'].values[0]
distance_Kennedy_parkway = calculate_distance(KSC_launch_site_lat, KSC_launch_site_long, Kennedy_parkway_lat, Kennedy_parkway_long)

Kennedy_coordinate = [Kennedy_parkway_lat, Kennedy_parkway_long]
distance_marker_Kennedy = folium.Marker(
  Kennedy_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Kennedy_parkway),
      )
  )

site_map.add_child(distance_marker_Kennedy)

coordinates = [[Kennedy_parkway_lat, Kennedy_parkway_long], [KSC_launch_site_lat, KSC_launch_site_long]]
lines=folium.PolyLine(locations=coordinates, weight=1)
site_map.add_child(lines)

Samuel_parkway_lat = 28.56357
Samuel_parkway_long = -80.57086
CCAFS_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].values[0]
CCAFS_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].values[0]
distance_Samuel_parkway = calculate_distance(CCAFS_launch_site_lat, CCAFS_launch_site_long, Samuel_parkway_lat, Samuel_parkway_long)

Samuel_coordinate = [Samuel_parkway_lat, Samuel_parkway_long]
distance_marker_Samuel = folium.Marker(
  Samuel_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Samuel_parkway),
      )
  )

site_map.add_child(distance_marker_Samuel)
site_map

Samuel_coordinates = [[Samuel_parkway_lat, Samuel_parkway_long], [CCAFS_launch_site_lat, CCAFS_launch_site_long]]
lines=folium.PolyLine(locations=Samuel_coordinates, weight=1)
site_map.add_child(lines)

In [55]:
#Distance to railways

Santa_rail_lat = 34.63462
Santa_rail_long = -120.62435
VAFB_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'VAFB SLC-4E', 'Lat'].values[0]
VAFB_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'VAFB SLC-4E', 'Long'].values[0]
distance_Santa_railway = calculate_distance(VAFB_launch_site_lat, VAFB_launch_site_long, Santa_rail_lat, Santa_rail_long)

Santa_coordinate = [Santa_rail_lat, Santa_rail_long]
distance_marker_Santa = folium.Marker(
  Santa_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Santa_railway),
      )
  )

site_map.add_child(distance_marker_Santa)

coordinates_Santa = [[Santa_rail_lat, Santa_rail_long], [VAFB_launch_site_lat, VAFB_launch_site_long]]
lines=folium.PolyLine(locations=coordinates_Santa, weight=1)
site_map.add_child(lines)

CCF_rail_lat = 28.57144
CCF_rail_long = -80.58543
CCAF_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].values[0]
CCAF_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].values[0]
distance_CCF_railway = calculate_distance(CCAF_launch_site_lat, CCAF_launch_site_long, CCF_rail_lat, CCF_rail_long)

CCF_coordinate = [CCF_rail_lat, CCF_rail_long]
distance_marker_CCF_rail = folium.Marker(
  CCF_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_CCF_railway),
      )
  )

site_map.add_child(distance_marker_CCF_rail)

coordinates_CCF_rail = [[CCF_rail_lat, CCF_rail_long], [CCAF_launch_site_lat, CCAF_launch_site_long]]
lines=folium.PolyLine(locations=coordinates_CCF_rail, weight=1)
site_map.add_child(lines)

In [58]:
#Distance to cities

Melbourne_lat = 28.10212
Melbourne_long = -80.63828
CCAF_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Lat'].values[0]
CCAF_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'CCAFS SLC-40', 'Long'].values[0]
distance_Melbourne = calculate_distance(CCAF_launch_site_lat, CCAF_launch_site_long, Melbourne_lat, Melbourne_long)

Melbourne_coordinate = [Melbourne_lat, Melbourne_long]
distance_marker_Melbourne = folium.Marker(
  Melbourne_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Melbourne),
      )
  )

site_map.add_child(distance_marker_Melbourne)

coordinates_Melbourne = [[Melbourne_lat, Melbourne_long], [CCAF_launch_site_lat, CCAF_launch_site_long]]
lines=folium.PolyLine(locations=coordinates_Melbourne, weight=1)
site_map.add_child(lines)

Santa_Maria_lat = 34.9512
Santa_Maria_long = -120.43762
VAFB_launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'VAFB SLC-4E', 'Lat'].values[0]
VAFB_launch_site_long = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'VAFB SLC-4E', 'Long'].values[0]
distance_Santa_Maria = calculate_distance(VAFB_launch_site_lat, VAFB_launch_site_long, Santa_Maria_lat, Santa_Maria_long)

Santa_Maria_coordinate = [Santa_Maria_lat, Santa_Maria_long]
distance_marker_Santa_Maria = folium.Marker(
  Santa_Maria_coordinate,
  icon=DivIcon(
      icon_size=(20,20),
      icon_anchor=(0,0),
      html='<div style="font-size: 12; color:red;"><b>%s</b></div>' % "{:10.2f} KM".format(distance_Santa_Maria),
      )
  )

site_map.add_child(distance_marker_Santa_Maria)

coordinates_Santa_Maria = [[Santa_Maria_lat, Santa_Maria_long], [VAFB_launch_site_lat, VAFB_launch_site_long]]
lines=folium.PolyLine(locations=coordinates_Santa_Maria, weight=1)
site_map.add_child(lines)